# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import pandas as pd
import logging
import json

from utilities import text_to_json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        responses = scoring_func(inputString)
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Writing driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [4]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [5]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"javascript arrays as objects.  possible duplicate: length of javascript object (ie. associative array) loop through javascript object    i'm trying to make an array, where each item has some name and value. the code above doesn't work. tryed to make an object, but it doesn't have a length property - no for loop.  is it possible to use arrays in this context?"

Here, we convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [6]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [7]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 259.38 ms


In [8]:
resp = predict_for(body)

INFO:model_driver:Predictions: [(11922383, 11922384, 0.4618297837830524), (750486, 750506, 0.10187547714586409), (5223, 6700, 0.0638246129572683), (85992, 86014, 0.04240792341938436), (4255472, 4255480, 0.026144231768798966), (684672, 684692, 0.02453308688418635), (126100, 4889658, 0.016040256778800346), (6491463, 6491621, 0.013490367187470405), (1584370, 1584377, 0.011515613015373344), (171251, 171256, 0.009396552126284062), (4616202, 4616273, 0.009032739115879984), (1885557, 1885660, 0.008370193039577081), (111102, 111111, 0.0071304530036020745), (2274242, 2274327, 0.005718410320988048), (7364150, 7364307, 0.0034374790002485965), (19590865, 19590901, 0.0034328185480412433), (14028959, 8716680, 0.00309644289435943), (840781, 840808, 0.002855357692136627), (20279484, 20279485, 0.002639894956650536), (695050, 695053, 0.002557405523119375), (8495687, 8495740, 0.002151802183642379), (7837456, 14853974, 0.0016722794698331785), (12953704, 12953750, 0.0014999543390222928), (4935632, 4935684,

Next, we move on to [building our docker image](03_Build_Image.ipynb).